<a href="https://colab.research.google.com/github/faith4hub/Exploratory_Data_Analysis/blob/main/SalaryPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Import Libraries**

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Built-in library
import itertools
import re
import json
from typing import Union, Optional, Sequence

# pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 2_000

##**Data Loading**

In [2]:
def load_data(*, filename: str, sep: str = ",") -> pd.DataFrame:
    """This is used to load the data.

    Params;
        filename (str): The filepath.
        sep (str, default=","): The separator. e.g ',', '\t', etc

    Returns:
        data (pd.DataFrame): The loaded dataframe.
    """
    data = pd.read_csv(filename, sep=sep)
    print(f"Shape of data: {data.shape}\n")
    return data

In [3]:
#load the data
fp = "salary.csv"
data = load_data(filename=fp)

data.head()

Shape of data: (32560, 15)



,Age,Workclass,Final_weight,Education,Education_num,Marital_status,Occupation,Relationship,Race,Sex,Capital_gain,Capital_loss,Hours_per_week,Country,Salary
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


##**Data Cleaning & Preprocessing**

In [4]:
#check no of values in a column and the data type
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Age             32560 non-null  int64 
 1   Workclass       32560 non-null  object
 2   Final_weight    32560 non-null  int64 
 3   Education       32560 non-null  object
 4   Education_num   32560 non-null  int64 
 5   Marital_status  32560 non-null  object
 6   Occupation      32560 non-null  object
 7   Relationship    32560 non-null  object
 8   Race            32560 non-null  object
 9   Sex             32560 non-null  object
 10  Capital_gain    32560 non-null  int64 
 11  Capital_loss    32560 non-null  int64 
 12  Hours_per_week  32560 non-null  int64 
 13  Country         32560 non-null  object
 14  Salary          32560 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [5]:
#no of missing value(s) in a column
data.isna().sum()

Age               0
Workclass         0
Final_weight      0
Education         0
Education_num     0
Marital_status    0
Occupation        0
Relationship      0
Race              0
Sex               0
Capital_gain      0
Capital_loss      0
Hours_per_week    0
Country           0
Salary            0
dtype: int64

In [6]:
#check the unique value(s) in the columns
for column in data.columns:
    print(f"{column}: Unique Values are: {data[column].unique()}")
    print("-------------------------------------------")

Age: Unique Values are: [50 38 53 28 37 49 52 31 42 30 23 32 40 34 25 43 54 35 59 56 19 39 20 45
 22 48 21 24 57 44 41 29 18 47 46 36 79 27 67 33 76 17 55 61 70 64 71 68
 66 51 58 26 60 90 75 65 77 62 63 80 72 74 69 73 81 78 88 82 83 84 85 86
 87]
-------------------------------------------
Workclass: Unique Values are: [' Self-emp-not-inc' ' Private' ' State-gov' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
-------------------------------------------
Final_weight: Unique Values are: [ 83311 215646 234721 ...  34066  84661 257302]
-------------------------------------------
Education: Unique Values are: [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
-------------------------------------------
Education_num: Unique Values are: [13  9  7 14  5 10 12 11  4 16 15  3  6  2  1  8]
-------------------------------------

In [7]:
#to remove white spaces in the columns
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [8]:
#to inspect further if other columns have "?"
columns_list = data.columns.tolist()

values = []
for i in columns_list : 
    num = round(data[data[i] == '?'].shape[0] * 100 / data.shape[0], 2)
    values.append(num)
    
result = pd.DataFrame({'feature': columns_list, 'num_missing_data': values})    
print(result)

           feature  num_missing_data
0              Age              0.00
1        Workclass              5.64
2     Final_weight              0.00
3        Education              0.00
4    Education_num              0.00
5   Marital_status              0.00
6       Occupation              5.66
7     Relationship              0.00
8             Race              0.00
9              Sex              0.00
10    Capital_gain              0.00
11    Capital_loss              0.00
12  Hours_per_week              0.00
13         Country              1.79
14          Salary              0.00


We can see that the features Occupation, Country and Workclass both have a value as "?". This doesnt give enough information for the column and as such we will be dropping the rows that have "?" value

In [9]:
#create a copy of the data to implement some changes only on the data.copy
data_copy=data.copy(deep=True)

In [10]:
#removing rows having "?" in Occupation or Workclass or Country
print("Before:", data.shape)
unclear_values=data_copy[(data_copy['Workclass'].str.contains('\?'))|(data_copy['Occupation'].str.contains('\?'))|(data_copy['Country'].str.contains('\?'))].index
data_copy.drop(unclear_values, inplace=True)
print("After:",data_copy.shape)


Before: (32560, 15)
After: (30161, 15)


In [11]:
#check the number of unique value(s) in the columns
for column in data_copy.columns:
    print(f"{column}: Number of unique values: {data_copy[column].nunique()}")
    print("-------------------------------------------")

Age: Number of unique values: 72
-------------------------------------------
Workclass: Number of unique values: 7
-------------------------------------------
Final_weight: Number of unique values: 20262
-------------------------------------------
Education: Number of unique values: 16
-------------------------------------------
Education_num: Number of unique values: 16
-------------------------------------------
Marital_status: Number of unique values: 7
-------------------------------------------
Occupation: Number of unique values: 14
-------------------------------------------
Relationship: Number of unique values: 6
-------------------------------------------
Race: Number of unique values: 5
-------------------------------------------
Sex: Number of unique values: 2
-------------------------------------------
Capital_gain: Number of unique values: 118
-------------------------------------------
Capital_loss: Number of unique values: 90
-------------------------------------------


In [12]:
#dividing the data into categorical and numerical data
num_data = data_copy.select_dtypes(include=[int, float])
cat_data = data_copy.select_dtypes(include=["O"])

num_data.shape, cat_data.shape

((30161, 6), (30161, 9))

In [13]:
#dropping some features
vars_to_drop = ["Final_weight","Education_num","Relationship", "Marital_status", "Race", "Country"]
data_copy.drop(columns=vars_to_drop, inplace=True)

data_copy.shape #confirm the features are dropped

(30161, 9)

In [16]:
# using pd.get_dummies for data prepocessing, where Salary >50k is 1 and below is 0
data_processed = pd.get_dummies(data_copy, drop_first=True)
print(data_processed.shape)

data_processed.head()

(30161, 40)


,Age,Capital_gain,Capital_loss,Hours_per_week,Workclass_Local-gov,Workclass_Private,Workclass_Self-emp-inc,Workclass_Self-emp-not-inc,Workclass_State-gov,Workclass_Without-pay,Education_11th,Education_12th,Education_1st-4th,Education_5th-6th,Education_7th-8th,Education_9th,Education_Assoc-acdm,Education_Assoc-voc,Education_Bachelors,Education_Doctorate,Education_HS-grad,Education_Masters,Education_Preschool,Education_Prof-school,Education_Some-college,Occupation_Armed-Forces,Occupation_Craft-repair,Occupation_Exec-managerial,Occupation_Farming-fishing,Occupation_Handlers-cleaners,Occupation_Machine-op-inspct,Occupation_Other-service,Occupation_Priv-house-serv,Occupation_Prof-specialty,Occupation_Protective-serv,Occupation_Sales,Occupation_Tech-support,Occupation_Transport-moving,Sex_Male,Salary_>50K
0,50,0,0,13,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,38,0,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
2,53,0,0,40,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,28,0,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,37,0,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# Check the correlation
corr_matrix = data_processed.corr()
corr_matrix

# # Visualize
# plt.figure(figsize=(35, 30))
# sns.heatmap(data=corr_matrix, annot=True)
# plt.title("Correlation of The Features", size=15)
# plt.show()

,Age,Capital_gain,Capital_loss,Hours_per_week,Workclass_Local-gov,Workclass_Private,Workclass_Self-emp-inc,Workclass_Self-emp-not-inc,Workclass_State-gov,Workclass_Without-pay,Education_11th,Education_12th,Education_1st-4th,Education_5th-6th,Education_7th-8th,Education_9th,Education_Assoc-acdm,Education_Assoc-voc,Education_Bachelors,Education_Doctorate,Education_HS-grad,Education_Masters,Education_Preschool,Education_Prof-school,Education_Some-college,Occupation_Armed-Forces,Occupation_Craft-repair,Occupation_Exec-managerial,Occupation_Farming-fishing,Occupation_Handlers-cleaners,Occupation_Machine-op-inspct,Occupation_Other-service,Occupation_Priv-house-serv,Occupation_Prof-specialty,Occupation_Protective-serv,Occupation_Sales,Occupation_Tech-support,Occupation_Transport-moving,Sex_Male,Salary_>50K
Age,1.000000,0.080154,0.060166,0.101599,0.068257,-0.210498,0.111040,0.150430,0.014804,0.015337,-0.087744,-0.055031,0.033400,0.024007,0.096016,0.017577,-0.016297,-0.003103,0.006946,0.074259,0.010759,0.096393,0.008390,0.059850,-0.093482,-0.010807,0.016109,0.111368,0.039446,-0.104207,-0.014959,-0.092834,0.018902,0.060614,0.005483,-0.029319,-0.018476,0.032397,0.081992,0.242000
Capital_gain,0.080154,1.000000,-0.032228,0.080432,-0.009623,-0.048179,0.096483,0.033325,-0.011611,-0.001758,-0.022410,-0.012555,-0.009367,-0.012222,-0.015692,-0.012327,-0.013483,-0.010191,0.038899,0.059681,-0.047221,0.047149,-0.000385,0.176065,-0.036122,-0.002547,-0.023033,0.057658,-0.012412,-0.024370,-0.027387,-0.042431,-0.007459,0.086311,-0.007546,0.010743,-0.010106,-0.018802,0.048811,0.221200
Capital_loss,0.060166,-0.032228,1.000000,0.052417,0.014725,-0.036391,0.030955,0.020219,-0.003727,-0.004711,-0.016818,-0.015440,-0.006072,-0.004092,-0.007513,-0.018182,0.002761,-0.009568,0.032819,0.049081,-0.030267,0.049305,-0.001245,0.048883,-0.021728,0.005183,-0.001811,0.049123,-0.011377,-0.023641,-0.019495,-0.043978,-0.011272,0.046737,-0.003446,0.009779,0.004340,-0.003247,0.047017,0.150050
Hours_per_week,0.101599,0.080432,0.052417,1.000000,0.001611,-0.095542,0.126254,0.087835,-0.032976,-0.014781,-0.106705,-0.049151,-0.015121,-0.017479,-0.009162,-0.022358,0.003931,0.018171,0.075469,0.064632,0.006436,0.065969,-0.013115,0.079401,-0.067670,-0.000382,0.044803,0.131571,0.093133,-0.055227,-0.004943,-0.182293,-0.046863,0.047379,0.023611,-0.005146,-0.021484,0.072791,0.231272,0.229480
Workclass_Local-gov,0.068257,-0.009623,0.014725,0.001611,1.000000,-0.456304,-0.052121,-0.081528,-0.057057,-0.005845,-0.025668,-0.008075,-0.011803,-0.015838,-0.010890,-0.008809,0.012353,-0.002946,0.043745,0.000356,-0.049362,0.130409,0.003114,-0.008045,-0.022964,-0.004686,-0.051368,-0.023846,-0.028574,-0.029522,-0.065778,-0.013241,-0.018721,0.160030,0.235905,-0.096769,-0.018776,0.004291,-0.043063,0.028668
Workclass_Private,-0.210498,-0.048179,-0.036391,-0.095542,-0.456304,1.000000,-0.323254,-0.505629,-0.353863,-0.036252,0.058789,0.027472,0.022915,0.031200,0.001364,0.027124,-0.010421,0.010105,-0.054876,-0.071587,0.065562,-0.113338,0.013200,-0.087214,0.014838,-0.029064,0.037323,-0.067425,-0.119009,0.093996,0.131283,0.071368,0.041029,-0.161750,-0.151357,0.057567,0.021654,0.029019,-0.066768,-0.117256
Workclass_Self-emp-inc,0.111040,0.096483,0.030955,0.126254,-0.052121,-0.323254,1.000000,-0.057756,-0.040420,-0.004141,-0.023757,-0.011956,-0.011095,-0.013348,-0.013069,-0.010571,-0.000890,-0.008384,0.038081,0.031722,-0.030679,0.015099,-0.007428,0.080405,-0.006376,-0.003320,-0.023402,0.128216,0.015856,-0.039864,-0.043493,-0.050679,-0.013263,0.006941,-0.022195,0.084805,-0.030796,-0.024130,0.084973,0.137643
Workclass_Self-emp-not-inc,0.150430,0.033325,0.020219,0.087835,-0.081528,-0.505629,-0.057756,1.000000,-0.063225,-0.006477,-0.018279,-0.014329,-0.000871,-0.006014,0.041853,-0.003650,-0.009046,-0.000172,-0.009940,0.016205,0.010698,-0.007883,-0.011618,0.077042,-0.020946,-0.005193,0.066844,0.018542,0.235065,-0.056337,-0.062315,-0.036313,-0.020745,0.010748,-0.039404,0.029381,-0.034812,-0.006628,0.107523,0.025569
W

##SPLIT THE DATA

In [38]:
from sklearn.model_selection import train_test_split


RANDOM_STATE, TEST_SIZE = 5, 0.2
TARGET = "Salary_>50K"

X = data_processed.drop(columns=[TARGET])
y = data_processed[TARGET]

X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

X_train.shape, X_validation.shape

((24128, 39), (6033, 39))

##**Build Regression Model**

In [39]:
from sklearn.linear_model import LinearRegression

# Instantiate
linear_model = LinearRegression()

# Fit
linear_model.fit(X_train, y_train)

# Make predictions
y_pred = linear_model.predict(X_validation)

In [40]:
y_pred

array([0.24331004, 0.37637696, 0.42982623, ..., 0.48747064, 0.17916969,
       0.34417925])

In [41]:
def create_result_dataframe(*, y_true: np.ndarray, y_pred: np.ndarray) -> pd.DataFrame:
    """This returns a DataFrame containing the true,
    predicted and error values."""
    # Convert predictions to a dataframe
    result = pd.DataFrame()  # Empty DF

    result["Salary>50k"] = y_true
    result["Predicted_salary"] = y_pred
    result["Error_in_predictions"] = result["Salary>50k"] - result["Predicted_salary"]

    return result

In [42]:
#where 0 = <50k and 1= >50k 
result = create_result_dataframe(y_true=y_validation, y_pred=y_pred)
result.sample(n=10, random_state=RANDOM_STATE)

,Salary>50k,Predicted_salary,Error_in_predictions
32436,0,0.132616,-0.132616
3790,0,0.395456,-0.395456
30989,0,0.148030,-0.148030
22942,1,0.269035,0.730965
13483,0,-0.123952,0.123952
26670,0,0.153689,-0.153689
30677,0,-0.035163,0.035163
8900,0,0.062437,-0.062437
29267,1,0.290072,0.709928
4481,0,0.166878,-0.166878


##**Evaluate the performance of the model**

In [43]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn import metrics


# Mean Squared Error (The lower, the better)
mse = mean_squared_error(y_true=y_validation, y_pred=y_pred, squared=True)

# Root Mean Squared Error (The lower, the better)
# Remember to use squared=False
rmse = mean_squared_error(y_true=y_validation, y_pred=y_pred, squared=False)

# Mean Absolute Error (The lower, the better)
mae = mean_absolute_error(y_true=y_validation, y_pred=y_pred)

# R Squared (The higher, the better. Max (best) value is 1)
R2 = r2_score(y_true=y_validation, y_pred=y_pred)

result_str = (
    "==================== Evaluation Metrics ===================="
        f"\nMean Squared Error (The lower, the better!): {round(mse, 3)}"
        f"\nRoot Mean Squared Error (The lower, the better!): {round(rmse, 3)}"
        f"\nMean Absolute Error (The lower, the better!): {round(mae, 3)}"
        "\n==========================================================="
        f"\nR Squared (The higher, the better!): {round(R2, 3)} "
)

print(result_str)

==================== Evaluation Metrics ====================
Mean Squared Error (The lower, the better!): 0.133
Root Mean Squared Error (The lower, the better!): 0.365
Mean Absolute Error (The lower, the better!): 0.289
R Squared (The higher, the better!): 0.275 
